In [152]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import PunktSentenceTokenizer
from nltk.stem import SnowballStemmer, LancasterStemmer

In [122]:
df = pd.read_csv("coffee_fix.csv")
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,agtron,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,50/73,9.0,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,54/77,9.0,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,54/70,9.0,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce..."
3,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRoast House\nEthiopia Suke Quto...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,53/79,8.0,8.0,9.0,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...
4,https://www.coffeereview.com/review/ethiopia-g...,\n\n\n94\n\n\nBig Creek Coffee Roasters\nEthi...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,48/70,9.0,9.0,9.0,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...


In [124]:
tokens = [word_tokenize(i) for i in df["desc_1"]]
t1 = tokens[0]

In [125]:
t1

['Evaluated',
 'as',
 'espresso',
 '.',
 'Sweet-toned',
 ',',
 'deeply',
 'rich',
 ',',
 'chocolaty',
 '.',
 'Vanilla',
 'paste',
 ',',
 'dark',
 'chocolate',
 ',',
 'narcissus',
 ',',
 'pink',
 'grapefruit',
 'zest',
 ',',
 'black',
 'cherry',
 'in',
 'aroma',
 'and',
 'cup',
 '.',
 'Plush',
 ',',
 'syrupy',
 'mouthfeel',
 ';',
 'resonant',
 ',',
 'flavor-saturated',
 'finish',
 '.',
 'In',
 'three',
 'parts',
 'milk',
 ',',
 'rich',
 'chocolate',
 'tones',
 'intensify',
 ',',
 'along',
 'with',
 'intimations',
 'of',
 'vanilla',
 'paste',
 'and',
 'black',
 'cherry',
 'in',
 'the',
 'short',
 'finish',
 'and',
 'floral-toned',
 'citrus',
 'zest',
 'in',
 'the',
 'long',
 '.']

In [126]:
count = 0
for i in df["desc_1"]:
    if "three parts milk" in i:
        #print(i)
        count += 1
print(count)

210


In [214]:
def process_content(description):
    l = []
    try:
        words = nltk.word_tokenize(description.lower().replace("-"," "))
        #print(words)
        tagged = nltk.pos_tag(words)
        #print(tagged)

        # All chunks
        chunk_gram = r"""Chunk: {<NNP>?<RB.?>*<VBD>*<JJ.?>*<NNP.?>*<NN.?>*<NN.?>*}"""
        chunk_parser = nltk.RegexpParser(chunk_gram)
        chunked = chunk_parser.parse(tagged)

        # Finding the finish description
        finish_gram = r"""Chunk: {<NNP>?<RB.?>*<VBD>*<VBN>*<JJ.?>*<NNP.?>*<NN.?>*<NN.?>*<IN>+<DT>+<JJ.?>*<NN>*}"""
        finish_parser = nltk.RegexpParser(finish_gram)
        finish_chunks = finish_parser.parse(tagged)

        # Finding the milk description
        
        # print(description)

        for chunk in chunked:
            if hasattr(chunk,"label"):
                if chunk.label() == "Chunk":
                    l.append(chunk)
            # print(chunk)
        #print("---Finish---")
        if len(finish_chunks) == 0:
            for chunk in chunked:
                if hasattr(chunk,"label"):
                    if chunk.label() == "Chunk":
                        for j in chunk:
                            print(j[0])
        # for chunk in finish_chunks:
        #     if hasattr(chunk,"label"):
        #         if chunk.label() == "Chunk":
        #             print(chunk)
        #             # for j in chunk:
        #             #     print(j[0])
        #print("---Finish done---")

    except Exception as e:
        print(str(e))

    return l

chunks = process_content(df["desc_1"][0])
chunks

[Tree('Chunk', [('espresso', 'NN')]),
 Tree('Chunk', [('sweet', 'NN')]),
 Tree('Chunk', [('toned', 'VBD')]),
 Tree('Chunk', [('deeply', 'RB'), ('rich', 'JJ')]),
 Tree('Chunk', [('chocolaty', 'NN')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('dark', 'JJ'), ('chocolate', 'NN')]),
 Tree('Chunk', [('narcissus', 'NN')]),
 Tree('Chunk', [('grapefruit', 'NN'), ('zest', 'NN')]),
 Tree('Chunk', [('black', 'JJ'), ('cherry', 'NN')]),
 Tree('Chunk', [('aroma', 'NN')]),
 Tree('Chunk', [('cup', 'NN')]),
 Tree('Chunk', [('plush', 'NN')]),
 Tree('Chunk', [('syrupy', 'JJ'), ('mouthfeel', 'NN')]),
 Tree('Chunk', [('resonant', 'NN')]),
 Tree('Chunk', [('flavor', 'NN')]),
 Tree('Chunk', [('saturated', 'VBD'), ('finish', 'JJ')]),
 Tree('Chunk', [('parts', 'NNS'), ('milk', 'NN')]),
 Tree('Chunk', [('rich', 'JJ'), ('chocolate', 'NN'), ('tones', 'NNS')]),
 Tree('Chunk', [('intimations', 'NNS')]),
 Tree('Chunk', [('vanilla', 'NN'), ('paste', 'NN')]),
 Tree('Chunk', [('black', 'JJ'

In [197]:
useless = ["milk","short","long","cup","flavor",
    "flavour","www","http","visit","information"
    "cafe","coffee","coffees","blend","shot"]

characteristic_names = ["finish","tone","tones","toned","mouthfeel","paste"
    "note","notes"]

def clean_chunk(chunk):
    grammar = [i[1] for i in chunk]
    words = [i[0] for i in chunk]
    for i in characteristic_names:
        for j in chunk:
            if j[0] == i:
                chunk.remove(j)
    for i in useless:
        if i in "".join(words):
            return None
    return chunk

In [198]:
clean = [clean_chunk(i) for i in chunks if clean_chunk(i) is not None]
clean

[Tree('Chunk', [('comprised', 'VBD')]),
 Tree('Chunk', [('panama', 'NN')]),
 Tree('Chunk', [('ethiopia', 'NN')]),
 Tree('Chunk', [('a.r.c.', 'NN')]),
 Tree('Chunk', [('motto', 'NN')]),
 Tree('Chunk', [('“', 'RB'), ('more', 'JJR')]),
 Tree('Chunk', [('specialty', 'NN')]),
 Tree('Chunk', [('”', 'NNP'), ('rigorously', 'RB')]),
 Tree('Chunk', [('vital', 'JJ'), ('elements', 'NNS')]),
 Tree('Chunk', [('sourcing', 'NN')]),
 Tree('Chunk', [('cafe', 'JJ'), ('environments', 'NNS')]),
 Tree('Chunk', [('water', 'NN'), ('quality', 'NN')]),
 Tree('Chunk', [('more', 'JJR'), ('information', 'NN')])]

In [199]:
stemmer = PorterStemmer()
stemmer = SnowballStemmer(language = "english")

In [208]:
def extract_characteristics(raw):
    characteristics = []
    for chunk in raw:
        grammar = [i[1] for i in chunk]
        words = [i[0] for i in chunk]
        clean = clean_chunk(chunk)
        if clean is None:
            continue

        if len(clean) == 0:
            continue
        
        if len(clean) == 1:
            if grammar[0] != "VBD":
                characteristics.append(stemmer.stem(words[0]))
            continue

        if all(i=="NN" for i in grammar):
            characteristics.append(stemmer.stem(words[0]))
            continue
        
        elif grammar[0].startswith('JJ') and grammar[1].startswith('NN'):
            characteristics.append(stemmer.stem(words[1]))
            continue

        elif grammar[0].startswith('RB') and grammar[1].startswith('JJ'):
            characteristics.append(stemmer.stem(words[1]))
            continue
    return np.array(characteristics)

extract_characteristics(chunks)

array(['panama', 'ethiopia', 'a.r.c.', 'motto', 'more', 'specialti',
       'element', 'sourc', 'environ', 'water', 'inform'], dtype='<U9')

In [209]:
df["characteristics"] = df["desc_1"].apply(lambda x:extract_characteristics(process_content(x)))

In [210]:
df.head()

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,characteristics
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,9.0,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"[espresso, sweet, rich, chocolati, vanilla, ch..."
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,9.0,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","[espresso, floral, honeysuckl, oak, chocol, th..."
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,9.0,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","[crispli, cocoa, blossom, date, rice, pepperco..."
3,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRoast House\nEthiopia Suke Quto...,92,Roast House,Ethiopia Suke Quto,"Spokane, Washington","Guji Zone, Oromia Region, south-central Ethiopia",Medium-Light,$19.00/16 ounces,November 2017,...,8.0,8.0,9.0,9.0,8.0,NaN,"Delicate, sweetly spice-toned. Pink peppercorn...",This coffee tied for the third-highest rating ...,Lavender-like flowers and hints of zesty pink ...,"[delic, peppercorn, date, myrrh, lavend, aroma..."
4,https://www.coffeereview.com/review/ethiopia-g...,\n\n\n94\n\n\nBig Creek Coffee Roasters\nEthi...,94,Big Creek Coffee Roasters,Ethiopia Gedeb Halo Beriti,"Hamilton, Montana","Gedeb District, Gedeo Zone, southern Ethiopia",Medium,$16.50/12 ounces,November 2017,...,9.0,9.0,9.0,9.0,8.0,NaN,"Deeply sweet, subtly pungent. Honey, pear, tan...",Southern Ethiopia coffees like this one are pr...,A deeply and generously lush cup saved from se...,"[sweet, pungent, honey, pear, tangerin, zest, ..."


In [223]:
all_characteristics = set([])
for i in df["characteristics"]:
    all_characteristics.update(i)

all_characteristics

{'access',
 'acid',
 'acrid',
 'addit',
 'adult',
 'agav',
 'akin',
 'alcohol',
 'aliv',
 'all',
 'allspic',
 'almond',
 'almondi',
 'almost',
 'aloewood',
 'also',
 'alyssum',
 'amber',
 'amond',
 'amplif',
 'amplifi',
 'anchor',
 'anis',
 'appeal',
 'appear',
 'appl',
 'apricot',
 'aprium',
 'aroma',
 'aromat',
 'array',
 'assess',
 'attract',
 'author',
 'autumn',
 'azalea',
 'back',
 'backdrop',
 'background',
 'bacon',
 'bake',
 'baker',
 'balanc',
 'ballast',
 'balm',
 'balsa',
 'banana',
 'barley',
 'barrel',
 'base',
 'bay',
 'bean',
 'beauti',
 'beef',
 'bergamot',
 'berri',
 'beverag',
 'big',
 'bight',
 'bing',
 'bit',
 'bitter',
 'bitter/tart',
 'bitterish',
 'bittersweet',
 'bizarr',
 'blackberri',
 'blackcurr',
 'blood',
 'bloom',
 'blossom',
 'blueberri',
 'bodi',
 'bold',
 'bosc',
 'bottom',
 'bouquet',
 'bourbon',
 'boysenberri',
 'brace',
 'brandi',
 'brazil',
 'breadfruit',
 'brew',
 'bright',
 'brilliant',
 'brisk',
 'brittl',
 'brothi',
 'brown',
 'bubblegum',
 'bu

In [249]:
df["coffee"] = df["name"] + "-" + df["roaster"]
df.head(3)

,slug,all_text,rating,roaster,name,location,origin,roast,est_price,review_date,...,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3,characteristics,coffee
0,https://www.coffeereview.com/review/sweety-esp...,\n\n\n95\n\n\nA.R.C.\n“Sweety” Espresso Blend...,95,A.R.C.,“Sweety” Espresso Blend,"Hong Kong, China",Panama; Ethiopia,Medium-Light,HKD $250/227 grams,November 2017,...,NaN,9.0,9.0,9.0,9.0,"Evaluated as espresso. Sweet-toned, deeply ric...",An espresso blend comprised of coffees from Pa...,A radiant espresso blend that shines equally i...,"[espresso, sweet, rich, chocolati, vanilla, ch...",“Sweety” Espresso Blend-A.R.C.
1,https://www.coffeereview.com/review/flora-blen...,\n\n\n94\n\n\nA.R.C.\nFlora Blend Espresso\n\...,94,A.R.C.,Flora Blend Espresso,"Hong Kong, China",Africa; Asia Pacific,Medium-Light,HKD $158/227 grams,November 2017,...,NaN,9.0,9.0,8.0,9.0,"Evaluated as espresso. Sweetly tart, floral-to...",An espresso blend comprised of coffees from Af...,"A floral-driven straight shot, amplified with ...","[espresso, floral, honeysuckl, oak, chocol, th...",Flora Blend Espresso-A.R.C.
2,https://www.coffeereview.com/review/ethiopia-s...,\n\n\n92\n\n\nRevel Coffee\nEthiopia Shakiso ...,92,Revel Coffee,Ethiopia Shakiso Mormora,"Billings, Montana","Guji Zone, southern Ethiopia",Medium-Light,$16.00/12 ounces,November 2017,...,8.0,8.0,9.0,8.0,NaN,"Crisply sweet, cocoa-toned. Lemon blossom, roa...",This coffee tied for the third-highest rating ...,"A gently spice-toned, floral- driven wet-proce...","[crispli, cocoa, blossom, date, rice, pepperco...",Ethiopia Shakiso Mormora-Revel Coffee


In [245]:
d = {}
for i in df["characteristics"]:
    for j in i:
        if j in d:
            d[j] += 1
        else:
            d[j] = 1

count_dict = {i:j for i,j in d.items() if j > 200}
count_dict

{'espresso': 315,
 'sweet': 1195,
 'rich': 415,
 'chocolati': 398,
 'chocol': 1558,
 'narcissus': 256,
 'grapefruit': 207,
 'cherri': 238,
 'aroma': 2255,
 'plush': 309,
 'syrupi': 536,
 'reson': 279,
 'floral': 487,
 'oak': 235,
 'gentl': 373,
 'crispli': 244,
 'cocoa': 1070,
 'date': 209,
 'delic': 230,
 'finish': 511,
 'hint': 433,
 'spice': 207,
 'crisp': 536,
 'acid': 1239,
 'zest': 334,
 'juici': 553,
 'structur': 691,
 'almond': 579,
 'sandalwood': 340,
 'fruit': 435,
 'high': 271,
 'apricot': 267,
 'magnolia': 291,
 'syrup': 202,
 'savori': 265,
 'full': 514,
 'deep': 224,
 'cedar': 761,
 'cut': 317,
 'tart': 468,
 'hazelnut': 358,
 'satini': 354,
 'flower': 351,
 'nut': 226,
 'underton': 256,
 'velveti': 293,
 'bittersweet': 209,
 'bright': 376}

In [246]:
one_hot = pd.DataFrame(columns = ["coffee"] + list(count_dict.keys()))
one_hot

,coffee,espresso,sweet,rich,chocolati,chocol,narcissus,grapefruit,cherri,aroma,...,cut,tart,hazelnut,satini,flower,nut,underton,velveti,bittersweet,bright


In [247]:
d = {"coffee":df["coffee"]}

for i in count_dict.keys():
    d[i] = df["characteristics"].apply(lambda x: 0 if i in x else 1)

In [248]:
one_hot = pd.DataFrame(d)
one_hot.head()

,coffee,espresso,sweet,rich,chocolati,chocol,narcissus,grapefruit,cherri,aroma,...,cut,tart,hazelnut,satini,flower,nut,underton,velveti,bittersweet,bright
0,“Sweety” Espresso Blend-A.R.C.,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,Flora Blend Espresso-A.R.C.,0,1,1,1,0,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
2,Ethiopia Shakiso Mormora-Revel Coffee,1,1,0,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
3,Ethiopia Suke Quto-Roast House,1,1,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1
4,Ethiopia Gedeb Halo Beriti-Big Creek Coffee Ro...,1,0,1,1,0,1,1,1,0,...,1,1,1,1,1,1,1,1,1,1


In [257]:
from sklearn.neighbors import NearestNeighbors

In [261]:
one_hot[one_hot.columns[1:]]

,espresso,sweet,rich,chocolati,chocol,narcissus,grapefruit,cherri,aroma,plush,...,cut,tart,hazelnut,satini,flower,nut,underton,velveti,bittersweet,bright
0,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
1,0,1,1,1,0,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,0,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
4,1,0,1,1,0,1,1,1,0,0,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,1,1,1,1,0,1,1,1,0,1,...,1,1,1,1,1,0,1,1,1,1
2278,1,1,1,0,1,1,1,1,0,1,...,1,1,1,1,1,1,1,1,1,1
2279,1,0,1,1,1,0,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
2280,1,0,1,0,0,1,1,1,0,1,...,1,0,0,1,1,1,1,1,1,1


In [274]:
X = one_hot[one_hot.columns[1:]]
nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

In [275]:
for i in indices:
    print(i,"\n")


[   0 2022  635  657] 

[   1  809  127 2253] 

[   2  992 1576    3] 

[   3 1847  442  383] 

[  4  31 253 504] 

[   5 1159  361  349] 

[   6  607 1213  513] 

[   7 1576  183  310] 

[   8 1719 1927 1443] 

[  9 465 122 472] 

[ 10  65  98 518] 

[  11  253    4 1337] 

[ 12 205 577 578] 

[ 13 296 479  84] 

[  14  433 2253  691] 

[  15 1658  789  837] 

[  16 1875  648 2253] 

[  17 1576 1474  992] 

[ 18 296 386 155] 

[ 19 837 789 578] 

[  20  563  313 1130] 

[  21 1130 1547 2031] 

[  22 1810 1130  573] 

[  23 1281  774  830] 

[  24  259 2018  750] 

[ 25 168 315 539] 

[  26  175 2095  873] 

[  27 2151  822  669] 

[  28 1643  903  852] 

[  29 1872  909 1849] 

[  30 1528    1  127] 

[ 31   4 191 138] 

[  32 1746  837  358] 

[ 33 262  72 817] 

[ 34 774 497 358] 

[ 35  31  10 129] 

[  36 1170 1852 1000] 

[ 37 495 386 514] 

[  38  863 1703  694] 

[  39 1576  543 2141] 

[ 40 666 671 116] 

[  41  791 1364  535] 

[ 42 168  71 436] 

[  43 1158 1989 2034] 

[ 44